# DOPP Exercise 2

## Introduction

## Load and check data

In [14]:
import pandas as pd
import glob
import os

folder_path = "../data/"
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
def clean_dataframe(file):
    df = pd.read_csv(file)  # Load CSV
    df = df.dropna()  # Remove NaN values
    
    # Check if 'Year' column exists and filter columns with 'Year' < 1990
    if 'Year' in df.columns:
        df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
        df = df[df['Year'] >= 1990] 
    
    return df

# Load, clean, and store each DataFrame in a dictionary
dataframes = {os.path.basename(file): clean_dataframe(file) for file in csv_files}

# Optionally, print the cleaned data for verification
for filename, df in dataframes.items():
    print(f"Cleaned DataFrame from {filename}:")
    print(df.head(), "\n")

Cleaned DataFrame from share-electricity-solar.csv:
        Entity Code  Year  Solar (% electricity)
0  Afghanistan  AFG  2000                    0.0
1  Afghanistan  AFG  2001                    0.0
2  Afghanistan  AFG  2002                    0.0
3  Afghanistan  AFG  2003                    0.0
4  Afghanistan  AFG  2004                    0.0 

Cleaned DataFrame from modern-renewable-energy-consumption.csv:
     Entity Code  Year  Wind Generation - TWh  Solar Generation - TWh  \
81  Algeria  DZA  1990                    0.0                     0.0   
82  Algeria  DZA  1991                    0.0                     0.0   
83  Algeria  DZA  1992                    0.0                     0.0   
84  Algeria  DZA  1993                    0.0                     0.0   
85  Algeria  DZA  1994                    0.0                     0.0   

    Geo Biomass Other - TWh  Hydro Generation - TWh  
81                      0.0                   0.135  
82                      0.0              